In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#

Found GPU at: /device:GPU:0


In [2]:
#-------------------------- Config.py --------------------------#
class Config:
#   TRAINSET_PATH = "/kaggle/input/cs229train/Frames"
  TRAINSET_PATH = "/kaggle/input/maxcs229train"
  SINGLE_TEST_PATH = "/kaggle/input/wolvesmancity122719hg/WolvesManCity122719H/clip19"
  TESTSET_PATH = "/kaggle/input/cs229test"
  BATCH_SIZE = 4
  EPOCHS = 3
  MODEL_PATH = "/kaggle/input/model3vids"
  PLAY_LEN = 30#seconds
  FPS = 2#frames per second
  FPP = PLAY_LEN * FPS#frames per play
  SEQ_LEN = 10#frames
#-------------------------- Imports --------------------------#
from os import listdir
from os.path import isfile, join, isdir
from PIL import Image
import numpy as np
import shelve
import cv2
import keras
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D
from keras.models import Sequential, load_model
from keras.layers import LayerNormalization
import matplotlib.pyplot as plt

In [3]:
#-------------------------- DataParsing.py --------------------------#
from os import listdir
from os.path import isfile, join, isdir
from PIL import Image
import numpy as np
import shelve
import cv2
def get_clips_by_stride(stride, frames_list, sequence_size):
    """ For data augmenting purposes.
    Parameters
    ----------
    stride : int
        The desired distance between two consecutive frames
    frames_list : list
        A list of sorted frames of shape 256 X 256
    sequence_size: int
        The size of the desired LSTM sequence
    Returns
    -------
    list
        A list of clips , 10 frames each
    """
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(np.copy(clip))
                cnt = 0
    return clips

def get_training_set():
    """
    Returns
    -------
    list
        A list of training sequences of shape (NUMBER_OF_SEQUENCES,SINGLE_SEQUENCE_SIZE,FRAME_WIDTH,FRAME_HEIGHT,1)
    """
    #####################################
    # cache = shelve.open(Config.CACHE_PATH)
    # return cache["datasetLSTM"]
    #####################################
    clips = []
    # loop over the training folders (Train000,Train001,..)
    print('Found %d games' %len(listdir(Config.TRAINSET_PATH)))
    for game in sorted(listdir(Config.TRAINSET_PATH))[:7]:
        if "New" in game or "Ars" in game or "Eve" in game:
            continue
        print('found %d clips for %s' %(len(listdir(join(Config.TRAINSET_PATH, game))), game))
        all_frames = []
        for clip in sorted(listdir(join(Config.TRAINSET_PATH, game))):
            if isdir(join(Config.TRAINSET_PATH, game, clip)):
#                 all_frames = []
                # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
                for frame in sorted(listdir(join(Config.TRAINSET_PATH, game, clip))):
                    if str(join(Config.TRAINSET_PATH, game, clip, frame))[-3:] == "tif":
                        img = Image.open(join(Config.TRAINSET_PATH, game, clip, frame)).resize((256, 256))
                        img = np.array(img, dtype=np.float32) / 256.0
                        all_frames.append(img)
                        
        # get the SEQ_LEN sequences from the list of images after applying data augmentation
        for stride in range(1, 3):
            print('all_frames.shape: ', np.array(all_frames).shape)
            clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=Config.SEQ_LEN))
#             print('clips.shape: ', np.array(clips).shape)
            input()
    return clips
#----------------------------- Parse Data From Max DB ----------------------------#
def get_max_training_set():
    """
    Returns
    -------
    list
        A list of training sequences of shape (NUMBER_OF_SEQUENCES,SINGLE_SEQUENCE_SIZE,FRAME_WIDTH,FRAME_HEIGHT,1)
    """
    #####################################
    # cache = shelve.open(Config.CACHE_PATH)
    # return cache["datasetLSTM"]
    #####################################
    clips = []
    # loop over the training folders (Train000,Train001,..)
    print('Found %d games' %len(listdir(Config.TRAINSET_PATH)))
    for game in sorted(listdir(Config.TRAINSET_PATH))[:3]:
        all_frames = np.load(join(Config.TRAINSET_PATH, game))
        print("loaded shape: ", all_frames.shape)
        print("reshaped shape: ", all_frames.reshape(-1, 256, 256).shape)
        all_frames = all_frames.reshape(-1, 256, 256)
        print("Loaded ", game)
        # get the SEQ_LEN sequences from the list of images after applying data augmentation
        for stride in range(1, 3):
#             print('all_frames.shape: ', np.array(all_frames).shape)
            clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=Config.SEQ_LEN))
#             print('clips.shape: ', np.array(clips).shape)
#             input()
    return clips
# get_max_training_set()
# get_training_set()

In [4]:
#-------------------------- UnsupervisedModels.py --------------------------#
import keras
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D
from keras.models import Sequential, load_model
from keras.layers import LayerNormalization
def get_lstm_model(reload_model=True):
    """
    Parameters
    ----------
    reload_model : bool
        Load saved model or retrain it
    """
    if not reload_model:
        return load_model(Config.MODEL_PATH,custom_objects={'LayerNormalization': LayerNormalization})
#     training_set = get_training_set()
    training_set = get_max_training_set()
    training_set = np.array(training_set)
    print('training_set.shape: ', training_set.shape)
    training_set = training_set.reshape(-1,Config.SEQ_LEN,256,256,1)
    print('training_set.shape after reshape: ', training_set.shape)
    seq = Sequential()
    seq.add(TimeDistributed(Conv2D(128, (11, 11), strides=4, padding="same"), batch_input_shape=(None, Config.SEQ_LEN, 256, 256, 1)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(64, (5, 5), strides=2, padding="same")))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(TimeDistributed(Conv2DTranspose(64, (5, 5), strides=2, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2DTranspose(128, (11, 11), strides=4, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(1, (11, 11), activation="sigmoid", padding="same")))
    print(seq.summary())
    seq.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=1e-4, decay=1e-5, epsilon=1e-6))
    seq.fit(training_set, training_set,
            batch_size=Config.BATCH_SIZE, epochs=Config.EPOCHS, shuffle=False)
    seq.save(Config.MODEL_PATH)
    return seq

def get_single_test(test_clip):
    test = np.zeros(shape=(Config.FPP, 256, 256, 1))
    cnt = 0
    for f in sorted(listdir(join(Config.TESTSET_PATH, test_clip))):
        if cnt == Config.FPP:
            break
        if str(join(join(Config.TESTSET_PATH, test_clip), f))[-3:] == "tif":
            img = Image.open(join(join(Config.TESTSET_PATH, test_clip), f)).resize((256, 256))
            img = np.array(img, dtype=np.float32) / 256.0
            test[cnt, :, :, 0] = img
            cnt += 1

    sz = test.shape[0] - Config.SEQ_LEN
    sequences = np.zeros((sz, Config.SEQ_LEN, 256, 256, 1))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((Config.SEQ_LEN, 256, 256, 1))
        for j in range(0, Config.SEQ_LEN):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip
    return sequences

In [5]:
#-------------------------- Train.py --------------------------#
import matplotlib.pyplot as plt

model = get_lstm_model(True)
print("saved model")

Found 19 games
loaded shape:  (80, 10, 256, 256)
reshaped shape:  (800, 256, 256)
Loaded  0.npy
loaded shape:  (80, 10, 256, 256)
reshaped shape:  (800, 256, 256)
Loaded  1.npy
loaded shape:  (57, 10, 256, 256)
reshaped shape:  (570, 256, 256)
Loaded  10.npy
training_set.shape:  (434, 10, 256, 256, 1)
training_set.shape after reshape:  (434, 10, 256, 256, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 10, 64, 64, 128)   15616     
_________________________________________________________________
layer_normalization (LayerNo (None, 10, 64, 64, 128)   256       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 32, 32, 64)    204864    
_________________________________________________________________
layer_normalization_1 (Layer (None, 10, 32, 32, 64)    128       
_________________________

PermissionDeniedError: /kaggle/input/model3vids/variables/variables_temp_0bb553d887a649bda0f9e036736f1874; Read-only file system [Op:SaveV2]

In [6]:
from keras.models import load_model
import matplotlib.pyplot as plt
from os import listdir
#-------------------------- Predict.py --------------------------#
test_clips = sorted(listdir(Config.TESTSET_PATH))
# model = get_lstm_model(reload_model=False)
#Test on NUM_TESTS randomly chosen clips from the testing set
# test_clips = random.sample(test_clips, Config.NUM_TESTS)
for test_clip in test_clips:
    print("Testing on " + test_clip)
    if test_clip[0] == '.':
        print("%s not a clip. Skipping..." %test_clip)
        continue
    test_sequences = get_single_test(test_clip)
    print("got test - shape: ", test_sequences.shape)
    # get the reconstruction cost of all the sequences
    reconstructed_sequences = model.predict(test_sequences,batch_size=4)
    sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(test_sequences[i],reconstructed_sequences[i])) for i in range(0,len(test_sequences))])
    sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
    sr = 1.0 - sa

    # plot the regularity scores
    plt.figure()
    plt.plot(sr)
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.savefig(test_clip +'.png')


Testing on clip101
got test - shape:  (50, 10, 256, 256, 1)


NameError: name 'model' is not defined

# 